<a href="https://colab.research.google.com/github/neennera/GIBBIN-Web-Scraping/blob/main/WebScaping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



---



# Open Chrome using Selenium

In [147]:
#Set up Chrome Driver
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:3 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu

In [148]:
#ระบุตำแหน่งของเว็บปลายทาง
destination = "https://esc.kmutt.ac.th/index.php"

In [149]:
#Open Chrome
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver',options=chrome_options)

In [150]:
#Open Website
driver = webdriver.Chrome('chromedriver',options=chrome_options)
driver.get(destination)

In [151]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

username = 'xxxxx'
password = 'xxxxx.'

username_input = driver.find_element("id","username")
username_input.send_keys(username)

pass_input = driver.find_element("id","password")
pass_input.send_keys(password)

search_box = driver.find_element("id","loginForm")
search_box.submit()



# user_input = driver.find_element("username","q")



---

# Web Scraping
- Get HTML source code using BeautifulSoup
- Read and transfer code to lecture data using HTMLParser

In [169]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [170]:
# driver.get(destination)
soup =  BeautifulSoup(driver.page_source, 'html.parser')

In [171]:
lecture = []

In [172]:
for i in range(0,5):
  get_div = driver.find_element('xpath','//*[@id="ttWrapper"]/table/tbody/tr[3]/td[' + str(i+2) + ']')
  lecture.append(get_div.get_attribute('innerHTML'))

In [173]:
from html.parser import HTMLParser

class MyHTMLParser(HTMLParser):
  def __init__(self):
    HTMLParser.__init__(self)
    HTMLParser.unwanted_tag = ['div','i','a','br']
    HTMLParser.lectuteInWeek = []
    HTMLParser.day = 0

  def GetLectuteInWeek(self, HTMLtext):
    for i in range(0,5):
      HTMLParser.lectuteInWeek.append([])
      parser.feed(HTMLtext[i])
      HTMLParser.day += 1
    return HTMLParser.lectuteInWeek
  
  def handle_data(self, data):
    # print("Encountered some data  :", data)
    HTMLParser.lectuteInWeek[HTMLParser.day].append(data)

  # def handle_starttag(self, tag, attrs):
  #   if not(tag in HTMLParser.unwanted_tag) :
  #     print("Encountered a start tag:", tag)

  # def handle_endtag(self, tag):
  #   if not(tag in HTMLParser.unwanted_tag) :
  #     print("Encountered an end tag :", tag)

parser = MyHTMLParser()
lectureInWeek = parser.GetLectuteInWeek(lecture)

# Analyse Data

In [176]:
temp = lectureInWeek.copy()

In [182]:
class lecture:
  def __init__(self, day=-1 , start=-1, end=-1, subject="none", className="none"):
    self.day = day
    self.start = start
    self.end = end
    self.className = className
    self.subject = subject

In [160]:
lectureData = []

In [195]:
for day in range(0,5):
  temp = lectureInWeek[day].copy()
  i=0
  while i< len(temp): 
    if temp[i][0:4] == "ESC ":  
      
      #get subject and class name
      subject = temp[i][0:3] + temp[i][4:-2]
      if subject[-1] == ".":
        subject = subject[:-1]
      className = temp[i+1]

      # get time
      timeIndex = temp.index(subject) +1
      time = temp[timeIndex]
      start = time[0:5]
      end = time[-5:]
      
      # remove read class
      temp.remove(subject)
      temp.remove(time)

      #append data
      print(str(day) + "  " + subject + " ; " + className + " ; ", start + " " + end)
      lectureData.append(lecture(day,start,end,subject,className))
      continue

    if temp[i] == "School Closed":
      print(str(day) + "  " + "School Closed")
      lectureData.append(lecture(day, "08:00" , "16:00", "School Closed"))
      break
    
    i+=1

0  School Closed
1  School Closed
2  ESC714 ; Divergence, curl and second derivatives ;  09:00 11:00
2  ESC623 ; solving your model ;  11:00 12:00
2  ESC623 ; solving your model ;  13:00 14:00
2  ESC711 ; Basic integration, integrability ;  14:00 16:00
3  ESC625 ; เตรียมสอบ TGAT ;  09:00 10:00
3  ESC713 ; 1st-order ODE (exact) ;  10:00 12:00
3  ESC712 ; Eigenvalues & eigenvectors ;  13:00 15:00
3  ESC621 ; Quiz ;  15:00 16:00
4  ESC621 ; ทฤษฎีกราฟ 1 ;  09:00 12:00
4  ESC625 ; การวางแผนครอบครัว ;  13:00 14:00
4  ESC642 ; ทักษะการส่งลูก(เสิร์ฟ) ;  14:00 15:00
4  ESC625 ; เตรียมสอบ TGAT ;  15:00 16:00
